In [1]:
import email
from email.policy import default
from __future__ import print_function
import base64

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

%config InlineBackend.figure_format = 'retina'

In [2]:
def setup():
    """
    Configuration to authenticate with API
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            # creds = flow.run_local_server(port=0)
            creds = flow.run_console()
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
            
    return creds

In [3]:
def get_labels(service, user_id='me'):
    """
    Returns list of email labels

    service: Resource instance for interacting with API
    user_id: string identifyin the gmail target user
    """
    results = service.users().labels().list(userId=user_id).execute()
    labels = results.get('labels', [])
    if not labels:
        print('No labels found.')
        return
    return labels

In [6]:
def list_messages(service, user_id='me', max_results=100):
    """
    Returns a list of email messages in the shape of ids

    service: Resource instance for interacting with API
    user_id: string identifyin the gmail target user
    """
    list_of_messages = service.users().messages().list(userId=user_id, includeSpamTrash=True, maxResults=max_results).execute()
    return list_of_messages


In [11]:
def get_message(service,  message_id, user_id='me', format='full'):
    """
    Gets a particular message/email's data by id

    service: Resource instance for interacting with API
    message_id: Unique message identifier. 
    user_id: String identifyin the gmail target user
    format: Specifies scope of contents in the returned message
    """
    e_mail = service.users().messages().get(userId=user_id, id=message_id, format='full').execute()
    return e_mail

In [59]:
def decode_message(data):
    """
    Decodes a base64 data message

    data: encoded data string
    """
    decoded_bytes = base64.b64decode(data)
    decoded_str = str(decoded_bytes, "utf-8")
    return decoded_str

In [13]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = setup()

    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        
        ## get labels 
        labels = get_labels(service, 'me')
        # print(labels)

        # get list of messages
        messages = list_messages(service, 'me', '100')
        # print(messages)
        message = get_message(service, '17f7542281582c8f', 'me', 'full')
        print(message)

        
        
        # print a particular message (email), iterate over the set of returned messages
        # focus on those messages that contain interesting labels (inbox and spam maybe?)
        # for the subset of messages, get the plain text part of it
        # decode the message


    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f'An error occurred: {error}')


In [14]:
main()

{'id': '17f7542281582c8f', 'threadId': '17f7542281582c8f', 'labelIds': ['CATEGORY_PROMOTIONS', 'UNREAD', 'INBOX'], 'snippet': 'We listened to your feedback and decided to extend the sunset date View this email in your browser Dear chinocr3@gmail.com, Thank you for all of the support and love that we received since the sunset', 'payload': {'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'chinocr3@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a1f:2b86:0:0:0:0:0 with SMTP id r128csp1657490vkr;        Thu, 10 Mar 2022 11:15:44 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'ABdhPJxxAwYcB6HRm2Nvn8FI0graMdTPBnkU242kdn1MpQtoVpDckcn0c8zHBMQYeMMp+GWp5c8U'}, {'name': 'X-Received', 'value': 'by 2002:a25:9001:0:b0:628:755e:f6bd with SMTP id s1-20020a259001000000b00628755ef6bdmr5259721ybl.481.1646939744726;        Thu, 10 Mar 2022 11:15:44 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1646939744; c